# Importing data/libraries

In [1]:
from MySQLConnect import mysql_engine
import pandas as pd
import numpy as np
from pandasql import sqldf

In [2]:
engine = mysql_engine() #creating an instance of the SQL connection engine from the central module.

In [3]:
uk_hp_index = pd.read_sql("select * from uk_house_price_index", engine)

In [4]:
uk_hp_index.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136125 entries, 0 to 136124
Data columns (total 57 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Date                    136125 non-null  object 
 1   RegionName              136125 non-null  object 
 2   AreaCode                136125 non-null  object 
 3   AveragePrice            136125 non-null  float64
 4   Index                   136125 non-null  float64
 5   IndexSA                 4764 non-null    float64
 6   1m%Change               135702 non-null  float64
 7   12m%Change              131325 non-null  float64
 8   AveragePriceSA          4764 non-null    float64
 9   SalesVolume             131658 non-null  float64
 10  DetachedPrice           129708 non-null  float64
 11  DetachedIndex           129708 non-null  float64
 12  Detached1m%Change       129319 non-null  float64
 13  Detached12m%Change      125064 non-null  float64
 14  SemiDetachedPrice   

## Fixing error in locations

> Locations with the region "United Kingdom" all have their state as "England". They'll be replaced here.

In [5]:
uk_hp_index.loc[uk_hp_index["RegionName"]=="United Kingdom", "state"] = np.nan
#Assigning NA to the state column where region name is "United Kingdom"

# Data Transformation

## Renaming columns for compatibility

>  Replacing "index" with "PriceIndex" to remove ambiguity, removing "%" from column names 

In [6]:
uk_hp_index.columns = uk_hp_index.columns.str.replace('Index', 'PriceIndex')
uk_hp_index.columns = uk_hp_index.columns.str.replace('%', '')

## Formatting the Date column

In [7]:
uk_hp_index[["Date"]].head()

,Date
0,01/01/2004
1,01/02/2004
2,01/03/2004
3,01/04/2004
4,01/05/2004


In [8]:
uk_hp_index["Date"] = pd.to_datetime(uk_hp_index["Date"], format="%d/%m/%Y") #converting "Date" from object to datetime.

In [9]:
max(uk_hp_index["Date"]) #records end in April 2023

Timestamp('2023-04-01 00:00:00')

In [10]:
min(uk_hp_index["Date"]) #records start in April 1968

Timestamp('1968-04-01 00:00:00')

In [11]:
pd.unique(uk_hp_index["Date"])

array(['2004-01-01T00:00:00.000000000', '2004-02-01T00:00:00.000000000',
       '2004-03-01T00:00:00.000000000', '2004-04-01T00:00:00.000000000',
       '2004-05-01T00:00:00.000000000', '2004-06-01T00:00:00.000000000',
       '2004-07-01T00:00:00.000000000', '2004-08-01T00:00:00.000000000',
       '2004-09-01T00:00:00.000000000', '2004-10-01T00:00:00.000000000',
       '2004-11-01T00:00:00.000000000', '2004-12-01T00:00:00.000000000',
       '2005-01-01T00:00:00.000000000', '2005-02-01T00:00:00.000000000',
       '2005-03-01T00:00:00.000000000', '2005-04-01T00:00:00.000000000',
       '2005-05-01T00:00:00.000000000', '2005-06-01T00:00:00.000000000',
       '2005-07-01T00:00:00.000000000', '2005-08-01T00:00:00.000000000',
       '2005-09-01T00:00:00.000000000', '2005-10-01T00:00:00.000000000',
       '2005-11-01T00:00:00.000000000', '2005-12-01T00:00:00.000000000',
       '2006-01-01T00:00:00.000000000', '2006-02-01T00:00:00.000000000',
       '2006-03-01T00:00:00.000000000', '2006-04-01

In [12]:
uk_hp_index.head() #records from Aberdeenshire only start in 2004.

,Date,RegionName,AreaCode,AveragePrice,PriceIndex,PriceIndexSA,1mChange,12mChange,AveragePriceSA,SalesVolume,...,New12mChange,NewSalesVolume,OldPrice,OldPriceIndex,Old1mChange,Old12mChange,OldSalesVolume,Country,state,county
0,2004-01-01,Aberdeenshire,S12000034,81693.66964,40.864214,NaN,NaN,NaN,NaN,388.0,...,NaN,103.0,81043.95084,40.883367,NaN,NaN,285.0,United Kingdom,Scotland,Aberdeenshire
1,2004-02-01,Aberdeenshire,S12000034,81678.76231,40.856757,NaN,-0.018248,NaN,NaN,326.0,...,NaN,107.0,80965.29542,40.843688,-0.097053,NaN,219.0,United Kingdom,Scotland,Aberdeenshire
2,2004-03-01,Aberdeenshire,S12000034,83525.09702,41.780317,NaN,2.260483,NaN,NaN,453.0,...,NaN,140.0,82903.23948,41.821302,2.393549,NaN,313.0,United Kingdom,Scotland,Aberdeenshire
3,2004-04-01,Aberdeenshire,S12000034,84333.67900,42.184780,NaN,0.968071,NaN,NaN,571.0,...,NaN,180.0,84003.99161,42.376586,1.327755,NaN,391.0,United Kingdom,Scotland,Aberdeenshire
4,2004-05-01,Aberdeenshire,S12000034,86379.95396,43.208353,NaN,2.426403,NaN,NaN,502.0,...,NaN,167.0,86222.73484,43.495852,2.641235,NaN,335.0,United Kingdom,Scotland,Aberdeenshire


> Creating Year and Month columns

In [13]:
uk_hp_index["Year"] = pd.DatetimeIndex(uk_hp_index["Date"]).year

In [14]:
uk_hp_index["Month"] = pd.DatetimeIndex(uk_hp_index["Date"]).month

## Removing country, county, and state level entries

> Checking for records that contain values from the high level locations

In [15]:
shires = pd.unique(uk_hp_index[uk_hp_index["RegionName"].str.contains("shire", case=False)]["RegionName"])

In [16]:
# Dropping "shires"

uk_hp_index = uk_hp_index.loc[~uk_hp_index["RegionName"].isin(shires)]

> The dataset was viewed in excel, and the following regions were excluded.

In [17]:
excluded_regions = ['East of England', 'England', 'England and Wales','Scotland', 'Wales','Northern Ireland',
                    'United Kingdom','Great Britain','South West','South East','North West','North East',
                    'Greater Manchester', 'London','Merseyside', 'Tyne and Wear','Scottish Borders']

In [18]:
excluded_regions.extend(pd.unique(uk_hp_index[uk_hp_index["RegionName"].str.contains("midlands", case=False)]["RegionName"]))

In [19]:
uk_hp_index = uk_hp_index.loc[~uk_hp_index["RegionName"].isin(excluded_regions)]
uk_hp_index.reset_index(drop=True, inplace=True)

## Imputing null counties

> There are 78 regions in which the county was not identified in the locations step. We will look over each of them in this step.

In [20]:
len(pd.unique(uk_hp_index[uk_hp_index["county"].isna()]["RegionName"]))

78

In [21]:
# Counting the number for each state

sqldf("select count(distinct RegionName), state from uk_hp_index where county is null group by state")

,count(distinct RegionName),state
0,72,England
1,3,Northern Ireland
2,3,Scotland


In [22]:
# Looking at the regions in Scotland

sqldf("select distinct(RegionName) from uk_hp_index where state == 'Scotland' and county is null")

,RegionName
0,City of Aberdeen
1,City of Edinburgh
2,Waverley


> Waverley was discovered to be in Surrey, England

In [23]:
#Replacing the state and county

uk_hp_index.loc[uk_hp_index["RegionName"]=="Waverley", "county"] = "Surrey"
uk_hp_index.loc[uk_hp_index["RegionName"]=="Waverley", "state"] = "England"

In [24]:
#Imputing the remaining two counties

uk_hp_index.loc[uk_hp_index["RegionName"]=="City of Aberdeen", "county"] = "Aberdeenshire"
uk_hp_index.loc[uk_hp_index["RegionName"]=="City of Edinburgh", "county"] = "City of Edinburgh"

> Next we look at the regions in England. The regions' respective counties were identified using Google searches and imputed manually.

In [25]:
sqldf("select distinct county from uk_hp_index where state == 'England'")["county"].values

array(['West Sussex', 'Derbyshire', 'Nottinghamshire', 'Kent', 'Suffolk',
       None, 'Essex', 'Hampshire', 'Bath and North East Somerset',
       'Bedford', 'Leicestershire', 'Blackburn with Darwen', 'Blackpool',
       'Lincolnshire', 'Bournemouth, Christchurch and Poole',
       'Bracknell Forest', 'Norfolk', 'City of Milton Keynes',
       'Brighton and Hove', 'Worcestershire', 'Hertfordshire',
       'Lancashire', 'Cambridgeshire', 'Staffordshire', 'Gloucestershire',
       'Oxfordshire', 'City of Bristol', 'Derby', 'Kingston upon Hull',
       'City of Peterborough', 'Cornwall', 'County Durham', 'Cumberland',
       'Darlington', 'Devon', 'Dorset', 'East Sussex', 'Surrey',
       'Tyne and Wear', 'Halton', 'Hartlepool', 'Somerset',
       'Isle of Wight', 'City of Leicester', 'Luton', 'Medway',
       'Middlesbrough', 'North Somerset', 'Northumberland',
       'Warwickshire', 'Redcar and Cleveland', 'Rutland',
       'Stockton-on-Tees', 'Swindon', 'Telford and Wrekin', 'Thurrock

In [26]:
sqldf("select distinct RegionName from uk_hp_index where state == 'England' and county is null")["RegionName"].values

array(['Barking and Dagenham', 'Barnet', 'Barnsley', 'Bexley',
       'Birmingham', 'Bolton', 'Bradford', 'Bromley', 'Broxtowe', 'Bury',
       'Calderdale', 'Camden', 'City of London', 'City of Nottingham',
       'City of Westminster', 'Coventry', 'Croydon', 'Doncaster',
       'Dudley', 'Ealing', 'Enfield', 'Greenwich', 'Hackney',
       'Hammersmith and Fulham', 'Haringey', 'Harrow', 'Hillingdon',
       'Hounslow', 'Inner London', 'Islington', 'Kensington and Chelsea',
       'Kingston upon Thames', 'Kirklees', 'Knowsley', 'Lambeth', 'Leeds',
       'Lewisham', 'Liverpool', 'Manchester', 'Merton',
       'Newcastle upon Tyne', 'Newham', 'North Tyneside', 'Oldham',
       'Outer London', 'Portsmouth', 'Reading', 'Redbridge',
       'Richmond upon Thames', 'Rochdale', 'Rotherham', 'Salford',
       'Sandwell', 'Sefton', 'Sheffield', 'Slough', 'Solihull',
       'Southampton', 'Southwark', 'St Helens', 'Stockport',
       'Stoke-on-Trent', 'Sunderland', 'Sutton', 'Tameside', 'Traffor

>Northumberland and Staffordshire each had one unidentified region, and those were placed directly in the dataset

In [27]:
uk_hp_index.loc[uk_hp_index["RegionName"]=="Newcastle upon Tyne", "county"] = "Northumberland"
uk_hp_index.loc[uk_hp_index["RegionName"]=="Stoke-on-Trent", "county"] = "Staffordshire"

> The remaining counties had their regions aggregated and batch imputed

In [28]:
# Greater London Boroughs
ldn = ['Barking and Dagenham', 'Barnet', 'Bexley', 'Bromley', 'Camden', 'City of London','City of Westminster',
      'Croydon', 'Ealing', 'Enfield', 'Greenwich', 'Hackney', 'Hammersmith and Fulham', 'Haringey', 'Harrow','Hillingdon',
      'Hounslow','Inner London', 'Islington', 'Kensington and Chelsea','Kingston upon Thames','Lambeth','Lewisham',
       'Merton','Newham', 'Outer London', 'Redbridge', 'Richmond upon Thames','Southwark', 'Sutton','Wandsworth']

In [29]:
# Yorkshire
yrk = ['Barnsley', 'Bradford', 'Calderdale', 'Doncaster', 'Kirklees', 'Leeds', 'Rotherham', 'Sheffield', 'Wakefield']

In [30]:
# Greater Manchester
manc = ['Bolton', 'Bury', 'Manchester', 'Oldham', 'Rochdale', 'Salford', 'Stockport', 'Tameside', 'Trafford', 'Wigan']

In [31]:
# Midlands
mid = ['Birmingham', 'Broxtowe', 'City of Nottingham', 'Coventry','Dudley', 'Sandwell', 'Solihull', 'Walsall',
       'Wolverhampton']

In [32]:
# Merseyside
mers = ['Knowsley','Liverpool','Sefton','St Helens','Wirral']

In [33]:
# Tyne and Wear
tyne = ['North Tyneside', 'Sunderland']

In [34]:
# Hampshire 
hamp = ['Portsmouth', 'Southampton']

In [35]:
# Berkshire
berk = ['Reading', 'Slough']

In [36]:
# Imputing
uk_hp_index.loc[uk_hp_index["RegionName"].isin(yrk), "county"] = "Yorkshire"
uk_hp_index.loc[uk_hp_index["RegionName"].isin(ldn), "county"] = "Greater London"
uk_hp_index.loc[uk_hp_index["RegionName"].isin(manc), "county"] = "Greater Manchester"
uk_hp_index.loc[uk_hp_index["RegionName"].isin(mid), "county"] = "Midlands"
uk_hp_index.loc[uk_hp_index["RegionName"].isin(mers), "county"] = "Merseyside"
uk_hp_index.loc[uk_hp_index["RegionName"].isin(tyne), "county"] = "Tyne and Wear"
uk_hp_index.loc[uk_hp_index["RegionName"].isin(hamp), "county"] = "Hampshire"
uk_hp_index.loc[uk_hp_index["RegionName"].isin(berk), "county"] = "Berkshire"

In [37]:
uk_hp_index.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105660 entries, 0 to 105659
Data columns (total 59 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   Date                    105660 non-null  datetime64[ns]
 1   RegionName              105660 non-null  object        
 2   AreaCode                105660 non-null  object        
 3   AveragePrice            105660 non-null  float64       
 4   PriceIndex              105660 non-null  float64       
 5   PriceIndexSA            0 non-null       float64       
 6   1mChange                105320 non-null  float64       
 7   12mChange               101844 non-null  float64       
 8   AveragePriceSA          0 non-null       float64       
 9   SalesVolume             105018 non-null  float64       
 10  DetachedPrice           102900 non-null  float64       
 11  DetachedPriceIndex      102900 non-null  float64       
 12  Detached1mChange        102594

## Selecting dates for further analysis

> Defining the selected dates function

In [38]:
def select_dates(df, dates, start, end): #creating a function to filter selected dates from the dataset.
    
    start_date = pd.to_datetime(start)
    end_date = pd.to_datetime(end)
    
    final = df[(dates>=start_date)&(dates<=end_date)]
    final.reset_index(drop=True, inplace=True)
    
    return final

In [39]:
len(pd.unique(select_dates(df = uk_hp_index, dates = uk_hp_index["Date"], start = "1968-01-01", 
                           end = "2004-12-31")["RegionName"]))
#between 1968 and 2004, there are 394 regions.

310

In [40]:
len(pd.unique(select_dates(df = uk_hp_index, dates = uk_hp_index["Date"], start = "2005-01-01", 
                           end = "2005-01-31")["RegionName"]))
#2005 has all 405 regions.

321

In [41]:
selected_records = select_dates(df = uk_hp_index, dates = uk_hp_index["Date"], start = "2005-01-01", end = "2023-12-31")
#Selecting records from 2006-2023 to ensure homogeneous regions.
#Records from 2005 were selected to be used later in the analysis process.

In [42]:
selected_records.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70620 entries, 0 to 70619
Data columns (total 59 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Date                    70620 non-null  datetime64[ns]
 1   RegionName              70620 non-null  object        
 2   AreaCode                70620 non-null  object        
 3   AveragePrice            70620 non-null  float64       
 4   PriceIndex              70620 non-null  float64       
 5   PriceIndexSA            0 non-null      float64       
 6   1mChange                70587 non-null  float64       
 7   12mChange               70488 non-null  float64       
 8   AveragePriceSA          0 non-null      float64       
 9   SalesVolume             69978 non-null  float64       
 10  DetachedPrice           67980 non-null  float64       
 11  DetachedPriceIndex      67980 non-null  float64       
 12  Detached1mChange        67980 non-null  float6

## Selecting features for further analysis


> Property type columns will be used to train the model

In [43]:
# selected_records = selected_records[["Date", "RegionName","state","Country",""]]

In [44]:
detached = selected_records.columns[selected_records.columns.str.contains("detached", case=False)]
# list of features of detached and semi-detached properties.

In [45]:
semi_detached = detached[detached.str.contains("semi", case=False)]
# list of features of semi-detached properties.

In [46]:
detached = detached[~detached.str.contains("semi", case=False)]
detached
# list of features of detached properties.

Index(['DetachedPrice', 'DetachedPriceIndex', 'Detached1mChange',
       'Detached12mChange'],
      dtype='object')

In [47]:
terraced = selected_records.columns[selected_records.columns.str.contains("terraced", case=False)]
terraced
# list of features of terraced properties.

Index(['TerracedPrice', 'TerracedPriceIndex', 'Terraced1mChange',
       'Terraced12mChange'],
      dtype='object')

In [48]:
flat = selected_records.columns[selected_records.columns.str.contains("flat", case=False)]
flat
# list of features of flat properties.

Index(['FlatPrice', 'FlatPriceIndex', 'Flat1mChange', 'Flat12mChange'], dtype='object')

In [49]:
identifiers = pd.concat([selected_records.iloc[:,[0, 58, 57, 1, 56, 55]]], axis=1)
identifiers
# subsetting the identifying features (Date, Month, Year, Region, state)

,Date,Month,Year,RegionName,county,state
0,2005-01-01,1,2005,Adur,West Sussex,England
1,2005-02-01,2,2005,Adur,West Sussex,England
2,2005-03-01,3,2005,Adur,West Sussex,England
3,2005-04-01,4,2005,Adur,West Sussex,England
4,2005-05-01,5,2005,Adur,West Sussex,England
...,...,...,...,...,...,...
70615,2022-12-01,12,2022,York,York,England
70616,2023-01-01,1,2023,York,York,England
70617,2023-02-01,2,2023,York,York,England
70618,2023-03-01,3,2023,York,York,England


## Exploring different property types 

In [50]:
# Creating a dataframe for the semidetached house type
semidetached_df = pd.concat([identifiers, selected_records[semi_detached]], axis=1)

In [51]:
semidetached_df.columns = semidetached_df.columns.str.replace('SemiDetached', '')
# removing the property type from the column name.
semidetached_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70620 entries, 0 to 70619
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Date        70620 non-null  datetime64[ns]
 1   Month       70620 non-null  int64         
 2   Year        70620 non-null  int64         
 3   RegionName  70620 non-null  object        
 4   county      69960 non-null  object        
 5   state       70620 non-null  object        
 6   Price       67980 non-null  float64       
 7   PriceIndex  67980 non-null  float64       
 8   1mChange    67980 non-null  float64       
 9   12mChange   67980 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int64(2), object(3)
memory usage: 5.4+ MB


In [52]:
semidetached_df.isna().sum()
#checking for NA values

Date             0
Month            0
Year             0
RegionName       0
county         660
state            0
Price         2640
PriceIndex    2640
1mChange      2640
12mChange     2640
dtype: int64

In [53]:
sqldf("select * from semidetached_df where Price is null").isna().sum()
#the index, month change, and year change are all null in the same places

Date             0
Month            0
Year             0
RegionName       0
county         660
state            0
Price         2640
PriceIndex    2640
1mChange      2640
12mChange     2640
dtype: int64

In [54]:
sqldf("select Distinct RegionName, state from semidetached_df where Price is null")

,RegionName,state
0,Antrim and Newtownabbey,Northern Ireland
1,Ards and North Down,Northern Ireland
2,Armagh City Banbridge and Craigavon,Northern Ireland
3,Belfast,Northern Ireland
4,Causeway Coast and Glens,Northern Ireland
5,City of London,England
6,Derry City and Strabane,Northern Ireland
7,Fermanagh and Omagh,Northern Ireland
8,Lisburn and Castlereagh,Northern Ireland
9,Mid Ulster,Northern Ireland


In [55]:
sqldf("select Distinct RegionName from semidetached_df where state = 'Northern Ireland'")

,RegionName
0,Antrim and Newtownabbey
1,Ards and North Down
2,Armagh City Banbridge and Craigavon
3,Belfast
4,Causeway Coast and Glens
5,Derry City and Strabane
6,Fermanagh and Omagh
7,Lisburn and Castlereagh
8,Mid Ulster
9,Mid and East Antrim


In [56]:
sqldf("select count(*) from semidetached_df where state = 'Northern Ireland'")

,count(*)
0,2420


In [57]:
sqldf("select count(*) from semidetached_df where state = 'Northern Ireland' and Price is null")

,count(*)
0,2420


In [58]:
sqldf("select count(*) from semidetached_df where RegionName = 'City of London'")

,count(*)
0,220


In [59]:
sqldf("select count(*) as count_NA from semidetached_df where RegionName = 'City of London' and price is null")

,count_NA
0,220


> All entries in City of London and Northern Ireland are null for Semi-Detached houses. The next step is to check whether it's the same for other house types

In [60]:
# Detached

detached_df = pd.concat([identifiers, selected_records[detached]], axis=1)
detached_df.columns = detached_df.columns.str.replace('Detached', '')
sqldf("select Distinct RegionName, state from detached_df where Price is null")

,RegionName,state
0,Antrim and Newtownabbey,Northern Ireland
1,Ards and North Down,Northern Ireland
2,Armagh City Banbridge and Craigavon,Northern Ireland
3,Belfast,Northern Ireland
4,Causeway Coast and Glens,Northern Ireland
5,City of London,England
6,Derry City and Strabane,Northern Ireland
7,Fermanagh and Omagh,Northern Ireland
8,Lisburn and Castlereagh,Northern Ireland
9,Mid Ulster,Northern Ireland


In [61]:
detached_df.isna().sum()

Date             0
Month            0
Year             0
RegionName       0
county         660
state            0
Price         2640
PriceIndex    2640
1mChange      2640
12mChange     2640
dtype: int64

In [62]:
sqldf("select count(*) from detached_df where state = 'Northern Ireland' and Price is null")

,count(*)
0,2420


In [63]:
sqldf("select count(*) as count_NA from detached_df where RegionName = 'City of London' and price is null")

,count_NA
0,220


In [64]:
# Terraced

terraced_df = pd.concat([identifiers, selected_records[terraced]], axis=1)
terraced_df.columns = terraced_df.columns.str.replace('Terraced', '')
sqldf("select Distinct RegionName, state from terraced_df where Price is null")

,RegionName,state
0,Antrim and Newtownabbey,Northern Ireland
1,Ards and North Down,Northern Ireland
2,Armagh City Banbridge and Craigavon,Northern Ireland
3,Belfast,Northern Ireland
4,Causeway Coast and Glens,Northern Ireland
5,City of London,England
6,Derry City and Strabane,Northern Ireland
7,Fermanagh and Omagh,Northern Ireland
8,Lisburn and Castlereagh,Northern Ireland
9,Mid Ulster,Northern Ireland


In [65]:
terraced_df.isna().sum()

Date             0
Month            0
Year             0
RegionName       0
county         660
state            0
Price         2613
PriceIndex    2613
1mChange      2613
12mChange     2613
dtype: int64

In [66]:
sqldf("select count(*) from terraced_df where state = 'Northern Ireland' and Price is null")

,count(*)
0,2420


In [67]:
sqldf("select count(*) as count_NA from terraced_df where RegionName = 'City of London' and price is null")

,count_NA
0,193


In [68]:
sqldf("select count(*) from terraced_df where RegionName = 'City of London' and price is not null")

,count(*)
0,27


In [69]:
# Flat

flat_df = pd.concat([identifiers, selected_records[flat]], axis=1)
flat_df.columns = flat_df.columns.str.replace('Flat', '')
sqldf("select Distinct RegionName, state from flat_df where Price is null")

,RegionName,state
0,Antrim and Newtownabbey,Northern Ireland
1,Ards and North Down,Northern Ireland
2,Armagh City Banbridge and Craigavon,Northern Ireland
3,Belfast,Northern Ireland
4,Causeway Coast and Glens,Northern Ireland
5,Derry City and Strabane,Northern Ireland
6,Fermanagh and Omagh,Northern Ireland
7,Lisburn and Castlereagh,Northern Ireland
8,Mid Ulster,Northern Ireland
9,Mid and East Antrim,Northern Ireland


In [70]:
flat_df.isna().sum()

Date             0
Month            0
Year             0
RegionName       0
county         660
state            0
Price         2420
PriceIndex    2420
1mChange      2420
12mChange     2420
dtype: int64

In [71]:
sqldf("select count(*) as count_NA from flat_df where state = 'Northern Ireland' and price is null")

,count_NA
0,2420


> The statement holds true for all property types except Flat, which has the values for the city of London. Records from Northern Ireland will be excluded  from the analysis, as well as records from City of London, as they are mostly absent.

## Creating categories and lagged indices for the property types

> Creating a category column will transform the data from a wide format to a long format. 

> Using lagged index columns can produce a stronger model, by incorporating the features used to calculate the index. This utilises nonlinear relationships between data, and also avoids discarding the data. Using the current index can lead to data leakage, as the index is derived from the price.

>The same can be said for 1-month and 12-month percentage changes, and these can also be used as lagged values.


In [72]:
# Semi-Detached

semidetached_df["PropertyType"] = "Semi-Detached"
#creating the property column

In [73]:
semidetached_df["LaggedIndex"] = semidetached_df["PriceIndex"].shift(1) #creating the lagged index column

In [74]:
semidetached_df[["Date","RegionName","LaggedIndex", "PriceIndex"]].head(24) #making sure the values correspond

,Date,RegionName,LaggedIndex,PriceIndex
0,2005-01-01,Adur,NaN,72.025650
1,2005-02-01,Adur,72.025650,69.089726
2,2005-03-01,Adur,69.089726,70.192094
3,2005-04-01,Adur,70.192094,70.663027
4,2005-05-01,Adur,70.663027,72.000041
5,2005-06-01,Adur,72.000041,71.517048
6,2005-07-01,Adur,71.517048,70.641371
7,2005-08-01,Adur,70.641371,71.744684
8,2005-09-01,Adur,71.744684,72.403430
9,2005-10-01,Adur,72.403430,73.148106


In [75]:
semidetached_df[["Date","RegionName","LaggedIndex", "PriceIndex"]].tail(24) #making sure the values correspond

,Date,RegionName,LaggedIndex,PriceIndex
70596,2021-05-01,York,133.503873,136.219886
70597,2021-06-01,York,136.219886,136.764258
70598,2021-07-01,York,136.764258,136.423833
70599,2021-08-01,York,136.423833,135.988140
70600,2021-09-01,York,135.988140,135.760647
70601,2021-10-01,York,135.760647,139.036744
70602,2021-11-01,York,139.036744,141.574610
70603,2021-12-01,York,141.574610,143.607417
70604,2022-01-01,York,143.607417,142.562875
70605,2022-02-01,York,142.562875,143.485458


In [76]:
# creating the lagged 1m and 12m change

semidetached_df["LaggedMonthChange"] = semidetached_df["1mChange"].shift(1)
semidetached_df["LaggedYearChange"] = semidetached_df["12mChange"].shift(12)

In [77]:
semidetached_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70620 entries, 0 to 70619
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Date               70620 non-null  datetime64[ns]
 1   Month              70620 non-null  int64         
 2   Year               70620 non-null  int64         
 3   RegionName         70620 non-null  object        
 4   county             69960 non-null  object        
 5   state              70620 non-null  object        
 6   Price              67980 non-null  float64       
 7   PriceIndex         67980 non-null  float64       
 8   1mChange           67980 non-null  float64       
 9   12mChange          67980 non-null  float64       
 10  PropertyType       70620 non-null  object        
 11  LaggedIndex        67979 non-null  float64       
 12  LaggedMonthChange  67979 non-null  float64       
 13  LaggedYearChange   67968 non-null  float64       
dtypes: dat

In [78]:
semidetached_df.drop(columns=["PriceIndex","1mChange", "12mChange"], inplace=True)

In [79]:
# Detached

detached_df["PropertyType"] = "Detached"
detached_df["LaggedIndex"] = detached_df["PriceIndex"].shift(1)
detached_df["LaggedMonthChange"] = detached_df["1mChange"].shift(1)
detached_df["LaggedYearChange"] = detached_df["12mChange"].shift(12)
detached_df.drop(columns=["PriceIndex","1mChange", "12mChange"], inplace=True)
detached_df.head()

,Date,Month,Year,RegionName,county,state,Price,PropertyType,LaggedIndex,LaggedMonthChange,LaggedYearChange
0,2005-01-01,1,2005,Adur,West Sussex,England,271270.7029,Detached,NaN,NaN,NaN
1,2005-02-01,2,2005,Adur,West Sussex,England,259423.6415,Detached,72.033817,-1.494210,NaN
2,2005-03-01,3,2005,Adur,West Sussex,England,263129.7082,Detached,68.887922,-4.367247,NaN
3,2005-04-01,4,2005,Adur,West Sussex,England,263037.4922,Detached,69.872039,1.428577,NaN
4,2005-05-01,5,2005,Adur,West Sussex,England,268621.0494,Detached,69.847552,-0.035046,NaN


In [80]:
# Terraced

terraced_df["PropertyType"] = "Terraced"
terraced_df["LaggedIndex"] = terraced_df["PriceIndex"].shift(1)
terraced_df["LaggedMonthChange"] = terraced_df["1mChange"].shift(1)
terraced_df["LaggedYearChange"] = terraced_df["12mChange"].shift(12)
terraced_df.drop(columns=["PriceIndex","1mChange", "12mChange"], inplace=True)
terraced_df.head()

,Date,Month,Year,RegionName,county,state,Price,PropertyType,LaggedIndex,LaggedMonthChange,LaggedYearChange
0,2005-01-01,1,2005,Adur,West Sussex,England,168151.0808,Terraced,NaN,NaN,NaN
1,2005-02-01,2,2005,Adur,West Sussex,England,161262.9939,Terraced,70.256619,-1.361034,NaN
2,2005-03-01,3,2005,Adur,West Sussex,England,163686.8852,Terraced,67.378649,-4.096368,NaN
3,2005-04-01,4,2005,Adur,West Sussex,England,164991.4247,Terraced,68.391396,1.503067,NaN
4,2005-05-01,5,2005,Adur,West Sussex,England,168582.6873,Terraced,68.936456,0.796973,NaN


In [81]:
# Flat

flat_df["PropertyType"] = "Flat"
flat_df["LaggedIndex"] = flat_df["PriceIndex"].shift(1)
flat_df["LaggedMonthChange"] = flat_df["1mChange"].shift(1)
flat_df["LaggedYearChange"] = flat_df["12mChange"].shift(12)
flat_df.drop(columns=["PriceIndex","1mChange", "12mChange"], inplace=True)
flat_df.head()

,Date,Month,Year,RegionName,county,state,Price,PropertyType,LaggedIndex,LaggedMonthChange,LaggedYearChange
0,2005-01-01,1,2005,Adur,West Sussex,England,129135.9120,Flat,NaN,NaN,NaN
1,2005-02-01,2,2005,Adur,West Sussex,England,123532.0211,Flat,80.581132,-1.605792,NaN
2,2005-03-01,3,2005,Adur,West Sussex,England,125076.0910,Flat,77.084290,-4.339529,NaN
3,2005-04-01,4,2005,Adur,West Sussex,England,125924.5936,Flat,78.047794,1.249935,NaN
4,2005-05-01,5,2005,Adur,West Sussex,England,128625.0053,Flat,78.577262,0.678389,NaN


## Joining the dataframes together in long format

> Here, the four dataframes are concatenated row-wise, and the records from 2005 are removed.

> The records from Northern Ireland and the London sub-regions are also removed.

In [82]:
hp_index_long = pd.concat([semidetached_df, detached_df, terraced_df, flat_df],axis=0) #row-wise concatenation
hp_index_long.reset_index(drop=True, inplace=True)

In [83]:
# capitalising the state and county columns, renaming price as AvgPrice.

hp_index_long.columns = hp_index_long.columns.str.replace('county','County')
hp_index_long.columns = hp_index_long.columns.str.replace('state','State')
hp_index_long.columns = hp_index_long.columns.str.replace('Price','AvgPrice')

In [84]:
hp_index_long.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 282480 entries, 0 to 282479
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   Date               282480 non-null  datetime64[ns]
 1   Month              282480 non-null  int64         
 2   Year               282480 non-null  int64         
 3   RegionName         282480 non-null  object        
 4   County             279840 non-null  object        
 5   State              282480 non-null  object        
 6   AvgPrice           272167 non-null  float64       
 7   PropertyType       282480 non-null  object        
 8   LaggedIndex        272163 non-null  float64       
 9   LaggedMonthChange  272163 non-null  float64       
 10  LaggedYearChange   272119 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int64(2), object(4)
memory usage: 23.7+ MB


In [85]:
hp_index_long = hp_index_long.sort_values(by=["RegionName","PropertyType","Date"])

In [86]:
hp_index_long = select_dates(df=hp_index_long, dates=hp_index_long["Date"], start="2006-01-01", end="2023-12-31")

### Removing Northern Ireland and the City of London

In [87]:
# Removing Northern Ireland records

hp_index_long = hp_index_long[~(hp_index_long["State"]=="Northern Ireland")]

In [88]:
hp_index_final = hp_index_long[~(hp_index_long["RegionName"]=="City of London")]

> Dropping the date column 

In [89]:
hp_index_final = hp_index_final.drop(columns=["Date"])

In [90]:
hp_index_final.isna().sum()

Month                0
Year                 0
RegionName           0
County               0
State                0
AvgPrice             0
PropertyType         0
LaggedIndex          0
LaggedMonthChange    0
LaggedYearChange     0
dtype: int64

In [91]:
hp_index_final.to_sql(name="modified_hp_data", con=engine, if_exists="replace",index=None)

257088

In [92]:
# uk_hp_index["Year"] = pd.DatetimeIndex(uk_hp_index["Date"]).year

In [93]:
# len(pd.unique(uk_hp_index["Year"]))

In [94]:
# uk_hp_index[(uk_hp_index["Year"]<=2022) & (uk_hp_index["Year"]>=2012)]

In [95]:
# uk_hp_index[uk_hp_index["Year"]==2023]